# 설명 가능한 인공지능 방법론 SHAP value 에 대하여

black-box 모델인 머신러닝의 예측값에 대한 설명 니즈가 커지고 있다.

이에맞추어 오늘은 XAI 방법론 중에 하나인 SHAP value 에 대해서 깊이 파보고자 한다.

경제학의 게임이론 분야에서 대가인 Lloyd Shapley(1951) 가 제안했던 Shapley Value의 개념을 머신러닝의 해석에 적용한 것이 SHAP Value 이다.

이러한 SHAP Value는 Su-In Lee 와 Scott Lundberg 가 2017년에 "A Unified Approach to Interpreting Model Predictions" 라는 제목의 논문으로 발표하였다.

Shapley value는 Efficiency, Dummy player, Symmetry, Strong monotonicity, Additivity 등의 지표로써 매력적인 성질들을 가지고 있다.

이에 따라 SHAP value 역시 변수들의 contribution에 대한 measure로써 좋은 성질들을 가지고 있다.

SHAP value에 대해 본격적으로 들어가기에 앞서, Shpley value에 대해 좀 더 알아보자.

## Shapley value

본래 Shapley Value는 팀플레이의 결과물로 나온 보상을 나눠가질때 각 팀원들에게 돌아갈 보상을 합리적으로 정하는데 그 목적이 있었다.

즉, 기여를 많이한 팀원에게 많은 보상이 돌아갈테고, 기여를 적게한 팀원에게 적은 보상이 돌아가게 될 것이다.

그리고 그러한 보상의 정도를 정량적으로 Shapley value를 통해 정할수 있는 것이다.

예를 들어보자. 3명의 팀원이 한 팀이 되어 상품을 판매한다고 해보자. 1달 후 상품이 100만개 팔렸다고 했을때 3명의 팀원들에게 돌아갈 보상은 어떻게 될까

가장 심플한 방식으로는 공산주의적 방식으로 3명 모두에게 전체 이익의 1/3씩 나눠주는 것이다.

하지만 이 팀원 5명이 각각 A,B,C 라고 할 때, A가 획기적인 방식을 고안해 상품을 비약적으로 많이 팔 수 있게 되었다고 쳐보자.

또한, 팀원 C는 한달간 몸이 아파 상품 판매에 거의 기여하지 못했다고 해보자. 이럴 경우에 어떻게 해야 합리적으로 팀원들에게 보상을 나눌 수 있을까

Shapley value는 가상의 실험을 통해 이 문제를 해결하고자 한다. 그 가상의 실험은 아래와 같다.

**A의 기여도 계산을 위한 가상의 비교 실험 셋팅**

$\emptyset$   vs.   {A}    $\quad$$\quad$$\quad$$\quad$ from {A,B,C} 

$\emptyset$   vs.   {A}    $\quad$$\quad$$\quad$$\quad$ from {A,C,B} 

{B}   vs.   {B,A}    $\quad$$\quad$$\quad$$\quad$ from {B,A,C} 

{C}   vs.   {C,A}    $\quad$$\quad$$\quad$$\quad$ from {C,A,B} 

{B,C}   vs.   {B,C,A}    $\quad$$\quad$$\quad$$\quad$ from {B,C,A} 

{C,B}   vs.   {C,B,A}    $\quad$$\quad$$\quad$$\quad$ from {C,B,A} 

위 실험 셋팅을 유심히 보면, 왼쪽 셋팅은 A가 없는 모든 경우들이며 오른쪽 셋팅은 왼쪽 셋팅에서 A가 추가로 들어간 경우들 임을 알 수 있다.

즉, 위 6가지 실험에서 오른쪽과 왼쪽의 결과값의 차이는 각 경우에서 **A의 기여도**로 볼 수 있다.

$\underline{결론적으로\, 말하면\, 이\,  A의 \, 기여도 \, 6개를 \, 평균낸다면\, 팀에서\, A의\, 총\, 기여도를\, 구할\, 수\, 있게 \,된다.}$

위 실험셋팅은 위에 표시한 바와 같이 순열로 쉽게 만들수 있다. 순열에서 A가 위치한 지점에 따라서 그 앞의 부분집합을 대조군으로 두는 방식으로 말이다.

예를들어, 첫번째와 두번째 실험에서의 순열 {A,B,C}, {A,C,B} 에서 순서상 A의 앞에 해당하는 부분집합은 공집합이므로 이와 대조비교한다.

그리고 세번째 실험의 경우 순열 {B,A,C} 에서 순서상 A의 앞에 해당하는 부분집합은 {B} 이므로 이와 대조비교하는 식이다.

이런식으로 계산해서 얻을 수 있는 이점은 실험에 참여하는 팀원의 개수에 따라 나누어 각각 평균한 뒤에 이들을 다시 평균냄으로써 팀원의 개수 조합에 의한 기여도의 inflation을 방지할 수 있다는 점이다.

위의 셋팅에서도 팀원이 0명인 경우, 1명인 경우, 2명인 경우 모두 2가지씩 실험이 진행되어 팀원의 명수에 따른 조합의 개수에 영향을 받지 않게 되었다.

이제 A의 Shapley Value를 계산해보자. v(S)를 S라는 집합의 기여도라고 하면, A의 Shapley Value는 아래와 같다.

$\Large{\phi_A = \frac{[v(A) - v(\emptyset)] + [v(A) - v(\emptyset)] + [v(B,A) - v(B)] + [v(C,A) - v(C)] + [v(B,C,A) - v(B,C)] + [v(C,B,A) - v(C,B)]}{6}}$

한편, 이는 이렇게도 표현할 수 있다.

$\large{\phi_A = \frac{1}{3}} [\frac{[v(A) - v(\emptyset)] + [v(A) - v(\emptyset)]}{2} + \frac{[v(B,A) - v(B)] + [v(C,A) - v(C)]}{2} + \frac{[v(B,C,A) - v(B,C)] + [v(C,B,A) - v(C,B)]}{2}]$

앞서 언급했듯, Shapley value는 구성원의 개수별로 평균을 낸 후에 이를 다시 평균한다.

위 식에서도 구성원이 0명인 case, 1명인 case, 2명인 case에 각각 A의 기여도를 평균하여 구한 후에 이를 다시 평균한 것을 알 수 있다.

물론 사실상 구성원이 0명인 case나 2명인 case는 두 실험이 같은 실험이긴 하다. 하지만 이렇게 인위적으로 순열을 통해 실험을 만들면 모든 경우에 대해서 예외없이 평균 기여도를 구할 수 있게 되며 단순히 구성원의 조합의 수가 많아지는 것에 의한 기여도 증폭을 방지할 수 있다.

즉, 구성원이 0명일 때, 1명일 때, 2명일 때 각각 A의 (대표)기여도를 산출한 뒤에 이를 평균하여 A의 총 기여도를 메기는 것이라고 볼 수 있다.

이러한 Shapley value를 좀 더 일반적인 식으로 표현하면 아래와 같다. (i 번째 구성원에 대한 Shaple value이다)

$\Large{\phi_i\ = \frac{1}{N!}}$ $\Large{\sum\limits_{S \subset {N - \text{{i}}}}}$  $\normalsize{|S|! (N - |S| - 1)!}$ $\normalsize{[v(S \cup \text{{i}}) - v(S) ]}$

## SHAP  (Application to ML)

이제 위 Shapley value를 머신러닝의 예측값 해석에 적용하는 방법에 대해 알아보자.